# Red Inicial

## Imports

In [5]:
from collections import deque
from enum import Enum
from functools import cache
from typing import Optional

import gymnasium as gym
import numpy as np
from gymnasium.spaces import Box, Dict

## Clases y Funciones

In [6]:
class Packet_Generator():
    def __init__(self, min_ip=0, max_ip=2000,
                 min_port=0, max_port=4000,
                 min_protocol=0, max_protocol=100,
                 min_size=0, max_size=2**16 - 1,
                 min_rate=0, max_rate=100):
        
        self.packet = Dict({
            "IP":  Box(low=min_ip, high=max_ip, shape=(), dtype=int),
            "PORT":  Box(low=min_port, high=max_port, shape=(), dtype=int),
            "PROTOCOL":  Box(low=min_protocol, high=max_protocol, shape=(), dtype=int),
            "SIZE":  Box(low=min_size, high=max_size, shape=(), dtype=int)
        })
        self.min_rate: int = min_rate
        self.max_rate: int = max_rate

    def generate_packet(self):
        return self.packet.sample()

    def generate_packets(self):
        num_packets: int = np.random.randint(self.min_rate, self.max_rate)
        return [self.generate_packet() for _ in range(num_packets)]


class DOS_Packet_Generator(Packet_Generator):
    def __init__(self,
                 min_ip=0, max_ip=2000,
                 min_port=0, max_port=4000,
                 min_protocol=0, max_protocol=100,
                 min_size=0, max_size=2**16 - 1,
                 min_rate=10, max_rate=100):
        
        ip = np.random.randint(min_ip, max_ip)
        super().__init__(ip, ip,
                         min_port, max_port,
                         min_protocol, max_protocol,
                         min_size, max_size,
                         min_rate, max_rate)


class DDOS_Packet_Generator(Packet_Generator):
    def __init__(self,
                 min_ip=0, max_ip=2000,
                 min_port=0, max_port=4000,
                 min_protocol=0, max_protocol=100,
                 min_size=2**10, max_size=2**16 - 1,
                 min_rate=100, max_rate=1000):
        
        super().__init__(min_ip, max_ip,
                         min_port, max_port,
                         min_protocol, max_protocol,
                         min_size, max_size,
                         min_rate, max_rate)


class PacketAttack(Enum):
    @staticmethod
    def new_set(description, weight, class_ref):
        return {
            "Description": description,
            "weight": weight,
            "class": class_ref
        }

    @staticmethod
    def not_implemented():
        raise NotImplementedError(f"Class not implemented")

    @classmethod
    @cache
    def weights(cls):
        attack_weights = []
        for attack in PacketAttack:
            attack_weights.append(attack.value["weight"])
        return np.array(attack_weights)
    
    # ----ENUM VALUES----
    DOS = new_set("Denial of Service", 1.0, DOS_Packet_Generator)
    DDOS = new_set("Distributed Denial of Service", 2.0, DDOS_Packet_Generator)

dos_gen = DOS_Packet_Generator()

print(dos_gen.generate_packets())

[{'IP': 17, 'PORT': 1060, 'PROTOCOL': 62, 'SIZE': 36356}, {'IP': 17, 'PORT': 89, 'PROTOCOL': 16, 'SIZE': 57266}, {'IP': 17, 'PORT': 681, 'PROTOCOL': 82, 'SIZE': 20907}, {'IP': 17, 'PORT': 2939, 'PROTOCOL': 69, 'SIZE': 24733}, {'IP': 17, 'PORT': 727, 'PROTOCOL': 16, 'SIZE': 62577}, {'IP': 17, 'PORT': 85, 'PROTOCOL': 57, 'SIZE': 23564}, {'IP': 17, 'PORT': 970, 'PROTOCOL': 65, 'SIZE': 47430}, {'IP': 17, 'PORT': 115, 'PROTOCOL': 99, 'SIZE': 10220}, {'IP': 17, 'PORT': 3911, 'PROTOCOL': 44, 'SIZE': 48737}, {'IP': 17, 'PORT': 3845, 'PROTOCOL': 12, 'SIZE': 6842}, {'IP': 17, 'PORT': 2585, 'PROTOCOL': 21, 'SIZE': 20596}, {'IP': 17, 'PORT': 3329, 'PROTOCOL': 33, 'SIZE': 49465}, {'IP': 17, 'PORT': 365, 'PROTOCOL': 55, 'SIZE': 18099}, {'IP': 17, 'PORT': 3235, 'PROTOCOL': 71, 'SIZE': 64332}, {'IP': 17, 'PORT': 1483, 'PROTOCOL': 92, 'SIZE': 52032}, {'IP': 17, 'PORT': 1538, 'PROTOCOL': 77, 'SIZE': 25929}, {'IP': 17, 'PORT': 3809, 'PROTOCOL': 32, 'SIZE': 60242}, {'IP': 17, 'PORT': 1747, 'PROTOCOL': 77,

In [7]:
class Acciones(Enum):
    REENVIAR = 0
    DESCARTAR = 2


class RouterEnv(gym.Env):

    def __init__(self, max_len=10, seed: Optional[int] = None):
        super(RouterEnv, self).__init__()

        self.max_len: int = max_len
        self.queue = deque(maxlen=max_len)
        self.rate = 5  
        self.attack_probability = 0.1  
        self.step_durations = []

        